<a href="https://colab.research.google.com/github/mmcncc/ipynb/blob/main/Z_Image_Turbo_4bit_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 单元格 1: 安装依赖
!pip install -U git+https://github.com/huggingface/diffusers git+https://github.com/Disty0/sdnq
!pip install flask pyngrok

In [ ]:
# 单元格 2: 加载模型（需要几分钟）
import torch
import diffusers
from sdnq import SDNQConfig
from sdnq.loader import apply_sdnq_options_to_model

pipe = diffusers.ZImagePipeline.from_pretrained(
    "Disty0/Z-Image-Turbo-SDNQ-uint4-svd-r32",
    torch_dtype=torch.float32,
    device_map="cuda"
)
pipe.transformer = apply_sdnq_options_to_model(pipe.transformer, use_quantized_matmul=True)
pipe.text_encoder = apply_sdnq_options_to_model(pipe.text_encoder, use_quantized_matmul=True)

print("模型加载完成！")

In [ ]:
# 单元格 3: 启动 API 服务
# 运行后会显示一个 Gradio 地址，复制到本地脚本中使用
import io
import base64
import random
import gradio as gr
import torch
import PIL.Image

# 图像生成函数，适配 Gradio 接口
def sdnq_image_generator(prompt: str, width: int, height: int, seed: int) -> PIL.Image.Image:
    print(f"收到请求: {prompt[:50]}...")

    # 生成图片
    image = pipe(
        prompt=prompt,
        height=height,
        width=width,
        num_inference_steps=9,
        guidance_scale=0.0,
        generator=torch.manual_seed(seed),
    ).images[0]

    print("图片生成成功！")
    return image

# 创建 Gradio 接口
iface = gr.Interface(
    fn=sdnq_image_generator,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="A vibrant, fantastical landscape..."),
        gr.Slider(minimum=256, maximum=1024, step=64, value=1024, label="Width"),
        gr.Slider(minimum=256, maximum=1024, step=64, value=1024, label="Height"),
        gr.Number(label="Seed", value=random.randint(0, 999999), precision=0)
    ],
    outputs=gr.Image(label="Generated Image", type="pil"),
    title="Z-Image-Turbo-SDNQ-uint4-svd-r32 Image Generator",
    description="Generate high-quality images with Z-Image-Turbo-SDNQ-uint4-svd-r32 model."
)

# 启动 Gradio 接口，并使用 share=True 创建公共链接
# Gradio 会自动处理外部访问，无需 ngrok
print("正在启动 Gradio 界面...")
iface.launch(share=True, debug=True)
print("Gradio 界面已启动！请查看上面的公共链接。")